In [38]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'01.10.2022'

In [39]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [3]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [40]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position']], 
              left_on='Player', 
              right_on='Name')

In [41]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position
0,Joel Embiid,PHI,HOU,33.0,C,"11,000",62.9,12.9,72.4,18.6,MAIN,Joel Embiid,C/UTIL
1,Jusuf Nurkic,POR,BKN,32.5,C,"7,000",44.6,10.6,53.2,34.0,MAIN,Jusuf Nurkic,C/UTIL
2,Anfernee Simons,POR,BKN,36.5,PG/SG,"6,400",41.7,10.1,49.3,46.4,MAIN,Anfernee Simons,PG/SG/G/UTIL
3,Kevin Durant,BKN,POR,37.0,SF/PF,"10,600",57.9,9.5,65.7,29.0,MAIN,Kevin Durant,SF/PF/F/UTIL
4,Hassan Whiteside,UTA,DET,28.0,C,"5,200",35.6,8.8,43.1,37.4,MAIN,Hassan Whiteside,C/UTIL


In [42]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position
0,Joel Embiid,PHI,HOU,33.0,C,11000.0,62.9,12.9,72.4,18.6,MAIN,Joel Embiid,"[C, UTIL]"
1,Jusuf Nurkic,POR,BKN,32.5,C,7000.0,44.6,10.6,53.2,34.0,MAIN,Jusuf Nurkic,"[C, UTIL]"
2,Anfernee Simons,POR,BKN,36.5,PG/SG,6400.0,41.7,10.1,49.3,46.4,MAIN,Anfernee Simons,"[PG, SG, G, UTIL]"
3,Kevin Durant,BKN,POR,37.0,SF/PF,10600.0,57.9,9.5,65.7,29.0,MAIN,Kevin Durant,"[SF, PF, F, UTIL]"
4,Hassan Whiteside,UTA,DET,28.0,C,5200.0,35.6,8.8,43.1,37.4,MAIN,Hassan Whiteside,"[C, UTIL]"


In [43]:
df.to_excel("final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
os.system("final_PlayerPool_ETR_"+str(today)+".xlsx")

0

# Create the Constraint Problem

Goal: Maximize DK Salary

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [60]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [61]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [62]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 8

In [63]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    
prob += get_position_sum(player_vars, df, 'PG') <= 3
prob += get_position_sum(player_vars, df, 'PG') >= 1
prob += get_position_sum(player_vars, df, 'SG') <= 3
prob += get_position_sum(player_vars, df, 'SG') >= 1
#prob += get_position_sum(player_vars, df, 'G') == 1
prob += get_position_sum(player_vars, df, 'SF') <= 3
prob += get_position_sum(player_vars, df, 'SF') >= 1
prob += get_position_sum(player_vars, df, 'PF') <= 3
prob += get_position_sum(player_vars, df, 'PF') >= 1
#prob += get_position_sum(player_vars, df, 'F') == 1
prob += get_position_sum(player_vars, df, 'C') <= 2
prob += get_position_sum(player_vars, df, 'C') >= 1
#prob += get_position_sum(player_vars, df, 'UTIL') == 1



In [64]:
# total salary constraint

prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [65]:
# finally, specify the goal
prob += pulp.lpSum([df['DK Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [66]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [67]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Points'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['C', 'UTIL'] Jusuf Nurkic POR 7000.0 44.6
['PG', 'SG', 'G', 'UTIL'] Anfernee Simons POR 6400.0 41.7
['SF', 'PF', 'F', 'UTIL'] Kevin Durant BKN 10600.0 57.9
['C', 'UTIL'] Hassan Whiteside UTA 5200.0 35.6
['SG', 'SF', 'F', 'G', 'UTIL'] Furkan Korkmaz PHI 4900.0 33.6
['PF', 'F', 'UTIL'] Tobias Harris PHI 8000.0 44.1
['SG', 'G', 'UTIL'] Ben McLemore POR 3300.0 21.8
['PG', 'G', 'UTIL'] Keifer Sykes IND 4600.0 26.0


(50000.0, 305.3)

In [16]:
df.dtypes

Player              object
Team                object
Opp                 object
Minutes            float64
DK Position         object
DK Salary          float64
DK Points          float64
DK Value           float64
DK Ceiling         float64
DK Own             float64
DK Slate            object
Name                object
Roster Position     object
dtype: object